![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Migrating from HNSW to SVS-VAMANA

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/06_hnsw_to_svs_migration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates how to migrate existing HNSW vector indices to SVS-VAMANA for improved memory efficiency while maintaining search quality.

## What You'll Learn

- How to assess your current HNSW index for migration
- Step-by-step migration from HNSW to SVS-VAMANA
- Memory usage comparison and cost analysis
- Search quality validation between HNSW and SVS-VAMANA
- Performance benchmarking and recall comparison
- Migration decision framework for production systems

## Prerequisites

- Redis Stack 8.2.0+ with RediSearch 2.8.10+
- Existing HNSW index with substantial data (1000+ documents recommended)
- High-dimensional vectors (768+ dimensions for best compression benefits)

## HNSW vs SVS-VAMANA

**HNSW (Hierarchical Navigable Small World):**
- Excellent search quality and recall
- Fast query performance
- Higher memory usage (stores full-precision vectors)
- Good for applications prioritizing search quality

**SVS-VAMANA:**
- Competitive search quality with compression
- Significant memory savings (50-75% reduction)
- Built-in vector compression (LeanVec, quantization)
- Ideal for large-scale deployments with cost constraints

## 📋 HNSW to SVS-VAMANA Migration Checklist

**PRE-MIGRATION:**
- ☐ Backup existing HNSW index data
- ☐ Test migration on staging environment
- ☐ Validate search quality with real queries
- ☐ Measure baseline HNSW performance metrics
- ☐ Plan rollback strategy
- ☐ Document current HNSW parameters (M, EF_construction, EF_runtime)

**MIGRATION:**
- ☐ Create SVS-VAMANA index with tested configuration
- ☐ Migrate data in batches during low-traffic periods
- ☐ Monitor memory usage and indexing progress
- ☐ Validate data integrity after migration
- ☐ Test search functionality thoroughly
- ☐ Compare recall metrics with baseline

**POST-MIGRATION:**
- ☐ Monitor search performance and quality
- ☐ Track memory usage and cost savings
- ☐ Update application configuration
- ☐ Document new SVS-VAMANA settings
- ☐ Clean up old HNSW index after validation period
- ☐ Update monitoring and alerting thresholds

**💡 HNSW-SPECIFIC TIPS:**
- HNSW indices are more complex to rebuild than FLAT
- Consider the impact on applications using EF_runtime tuning
- SVS-VAMANA may have different optimal query parameters
- Test with your specific HNSW configuration (M, EF values)
- Monitor for 48-72 hours before removing HNSW index
- Keep compression settings documented for future reference

## 📦 Installation & Setup

This notebook uses **RedisVL vectorizers** for generating embeddings and **Redis Stack** for vector search.

**Requirements:**
- Redis Stack 8.2.0+ with RediSearch 2.8.10+ (for SVS-VAMANA support)
- redisvl>=0.11.0 (required for SVS-VAMANA migration features and vectorizers)
- redis-py>=6.4.0 (required for compatibility with RedisVL 0.11.0+)
- numpy (for vector operations)

**⚠️ Important:** If you encounter Redis connection errors, upgrade redis-py: `pip install -U "redis>=6.4.0"`

### Install Packages

In [1]:
%pip install git+https://github.com/redis/redis-vl-python.git "redis>=6.4.0" "numpy>=1.21.0" "sentence-transformers>=2.2.0"

/Users/nitin.kanukolanu/workspace/redis-vl-python/.venv/bin/python3: No module named pip
Note: you may need to restart the kernel to use updated packages.


### Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings and full text fields. **We need to have a Redis
instance available.**

#### Local Redis
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [20]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### Alternative Redis Access (Cloud, Docker, other)
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [1]:
import os
import json
import numpy as np
import time
from typing import List, Dict, Any

# Redis and RedisVL imports
import redis
from redisvl.index import SearchIndex
from redisvl.query import VectorQuery
from redisvl.redis.utils import array_to_buffer, buffer_to_array
from redisvl.utils import CompressionAdvisor
from redisvl.redis.connection import supports_svs

# RedisVL Vectorizer imports
from redisvl.utils.vectorize import HFTextVectorizer

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

print("📚 Libraries imported successfully!")

📚 Libraries imported successfully!


## Step 1: Verify Redis and SVS Support

First, let's ensure Redis Stack is running and supports SVS-VAMANA.

In [2]:
# Test Redis connection and SVS support
try:
    client = redis.Redis.from_url(REDIS_URL)
    client.ping()
    print("✅ Redis connection successful")
    
    # Check Redis version
    redis_info = client.info()
    redis_version = redis_info['redis_version']
    print(f"📊 Redis version: {redis_version}")
    
    # Check SVS support
    if supports_svs(client):
        print("✅ SVS-VAMANA supported")
    else:
        print("❌ SVS-VAMANA not supported")
        print("Please ensure you're using Redis Stack 8.2.0+ with RediSearch 2.8.10+")
        
except Exception as e:
    print(f"❌ Redis connection failed: {e}")
    print("Please ensure Redis Stack is running on localhost:6379")

✅ Redis connection successful
📊 Redis version: 8.2.2
✅ SVS-VAMANA supported


## Step 2: Load Sample Data

We'll use the movie dataset to demonstrate the migration process.

In [3]:
# Load the movies dataset
with open('resources/movies.json', 'r') as f:
    movies_data = json.load(f)

print(
    f"📽️ Loaded {len(movies_data)} movie records",
    f"Sample movie: {movies_data[0]['title']}",
    f"Genres available: {set(movie['genre'] for movie in movies_data)}",
    sep="\n"
)

# Configuration for demonstration  
dims = 768  # Using all-mpnet-base-v2 model (768 dimensions)
num_docs = len(movies_data)  # Use actual dataset size

print(
    f"\n🔧 Configuration:",
    f"Vector dimensions: {dims}",
    f"Dataset size: {num_docs} movie documents",
    f"Vectorizer: RedisVL HFTextVectorizer",
    sep="\n"
)

📽️ Loaded 20 movie records
Sample movie: Explosive Pursuit
Genres available: {'comedy', 'action'}

🔧 Configuration:
Vector dimensions: 768
Dataset size: 20 movie documents
Vectorizer: RedisVL HFTextVectorizer


## Step 3: Create HNSW Index

First, we'll create an HNSW index with typical production settings.

In [4]:
# Create HNSW schema with production-like settings
hnsw_schema = {
    "index": {
        "name": "hnsw_demo_index",
        "prefix": "demo:hnsw:",
    },
    "fields": [
        {"name": "movie_id", "type": "tag"},
        {"name": "title", "type": "text"},
        {"name": "genre", "type": "tag"},
        {"name": "rating", "type": "numeric"},
        {"name": "description", "type": "text"},
        {
            "name": "embedding",
            "type": "vector",
            "attrs": {
                "dims": dims,
                "algorithm": "hnsw",
                "datatype": "float32",
                "distance_metric": "cosine",
                "m": 16,  # Number of bi-directional links for each node
                "ef_construction": 200,  # Size of dynamic candidate list
                "ef_runtime": 10  # Size of dynamic candidate list during search
            }
        }
    ]
}

print("Creating HNSW index with optimized settings...")
hnsw_index = SearchIndex.from_dict(hnsw_schema, redis_url=REDIS_URL)
hnsw_index.create(overwrite=True)
print(f"✅ Created HNSW index: {hnsw_index.name}")

# Display HNSW configuration
print(
    "\n🔧 HNSW Configuration:",
    f"M (connections per node): 16",
    f"EF Construction: 200",
    f"EF Runtime: 10",
    f"Distance metric: cosine",
    f"Data type: float32",
    sep="\n"
)

Creating HNSW index with optimized settings...
✅ Created HNSW index: hnsw_demo_index

🔧 HNSW Configuration:
M (connections per node): 16
EF Construction: 200
EF Runtime: 10
Distance metric: cosine
Data type: float32


## Step 4: Generate Embeddings and Load HNSW Index

Generate embeddings for movie descriptions and populate the HNSW index.

In [5]:
# Generate embeddings using RedisVL vectorizers
print("🔄 Generating embeddings for movie descriptions...")

descriptions = [movie['description'] for movie in movies_data]

# Use RedisVL HFTextVectorizer
print("🚀 Using RedisVL HFTextVectorizer...")
vectorizer = HFTextVectorizer(
    model="sentence-transformers/all-mpnet-base-v2",  # 768 dimensions
)

# Generate embeddings using RedisVL vectorizer
embeddings = vectorizer.embed_many(descriptions)
embeddings = np.array(embeddings, dtype=np.float32)

print(f"✅ Generated {len(embeddings)} real embeddings using RedisVL HFTextVectorizer")
print(f"📊 Embedding shape: {embeddings.shape}")

🔄 Generating embeddings for movie descriptions...
🚀 Using RedisVL HFTextVectorizer...
12:13:05 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
12:13:05 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
✅ Generated 20 real embeddings using RedisVL HFTextVectorizer
📊 Embedding shape: (20, 768)


In [6]:
# Prepare data for loading into HNSW index
sample_data = []
for i, movie in enumerate(movies_data):
    sample_data.append({
        'movie_id': str(movie['id']),
        'title': movie['title'],
        'genre': movie['genre'],
        'rating': movie['rating'],
        'description': movie['description'],
        'embedding': array_to_buffer(embeddings[i].astype(np.float32), dtype='float32')
    })

print(f"📦 Prepared {len(sample_data)} documents for indexing")

📦 Prepared 20 documents for indexing


In [7]:
# Load data into HNSW index
print("📥 Loading data into HNSW index...")
batch_size = 100  # Process in batches

for i in range(0, len(sample_data), batch_size):
    batch = sample_data[i:i+batch_size]
    hnsw_index.load(batch)
    print(f"  Loaded {min(i+batch_size, len(sample_data))}/{len(sample_data)} documents")

# Wait for indexing to complete
print("⏳ Waiting for HNSW indexing to complete...")
time.sleep(5)  # HNSW indexing takes longer than FLAT

hnsw_info = hnsw_index.info()
print(
    f"\n✅ HNSW index loaded with {hnsw_info['num_docs']} documents",
    f"Index size: {hnsw_info.get('vector_index_sz_mb', 'N/A')} MB",
    f"Indexing time: ~5 seconds (HNSW graph construction)",
    sep="\n"
)

📥 Loading data into HNSW index...
  Loaded 20/20 documents
⏳ Waiting for HNSW indexing to complete...

✅ HNSW index loaded with 20 documents
Index size: 3.2259750366210938 MB
Indexing time: ~5 seconds (HNSW graph construction)


## Step 5: Get Compression Recommendation

Use the CompressionAdvisor to get optimal SVS-VAMANA settings for our data.

In [8]:
# Get compression recommendation
print("🔍 Analyzing data for optimal compression settings...")

# Get recommendations for different priorities
memory_config = CompressionAdvisor.recommend(dims=dims, priority="memory")
balanced_config = CompressionAdvisor.recommend(dims=dims, priority="balanced")
performance_config = CompressionAdvisor.recommend(dims=dims, priority="performance")

print(
    "\n📊 Compression Recommendations:",
    "",
    "🗜️ Memory Priority:",
    f"  Algorithm: {memory_config.algorithm}",
    f"  Compression: {memory_config.compression if hasattr(memory_config, 'compression') else 'None'}",
    f"  Datatype: {memory_config.datatype}",
    f"  Dimensions: {dims} → {memory_config.reduce if hasattr(memory_config, 'reduce') else dims}",
    "",
    "⚖️ Balanced Priority:",
    f"  Algorithm: {balanced_config.algorithm}",
    f"  Compression: {balanced_config.compression if hasattr(balanced_config, 'compression') else 'None'}",
    f"  Datatype: {balanced_config.datatype}",
    f"  Dimensions: {dims} → {balanced_config.reduce if hasattr(balanced_config, 'reduce') else dims}",
    "",
    "⚡ Performance Priority:",
    f"  Algorithm: {performance_config.algorithm}",
    f"  Compression: {performance_config.compression if hasattr(performance_config, 'compression') else 'None'}",
    f"  Datatype: {performance_config.datatype}",
    f"  Dimensions: {dims} → {performance_config.reduce if hasattr(performance_config, 'reduce') else dims}",
    sep="\n"
)

# Select configuration (using memory priority for maximum savings)
selected_config = memory_config
# Use reduce if it exists and is not None, otherwise use original dims
target_dims = selected_config.reduce if (hasattr(selected_config, 'reduce') and selected_config.reduce is not None) else dims
target_dtype = selected_config.datatype

print(
    f"\n✅ Selected configuration: Memory Priority",
    f"Expected memory reduction: ~{((dims - target_dims) / dims * 100):.1f}% from dimension reduction",
    f"Additional savings from {selected_config.datatype} compression",
    sep="\n"
)

🔍 Analyzing data for optimal compression settings...

📊 Compression Recommendations:

🗜️ Memory Priority:
  Algorithm: svs-vamana
  Compression: LVQ4
  Datatype: float32
  Dimensions: 768 → None

⚖️ Balanced Priority:
  Algorithm: svs-vamana
  Compression: LVQ4x4
  Datatype: float32
  Dimensions: 768 → None

⚡ Performance Priority:
  Algorithm: svs-vamana
  Compression: LVQ4x4
  Datatype: float32
  Dimensions: 768 → None

✅ Selected configuration: Memory Priority
Expected memory reduction: ~0.0% from dimension reduction
Additional savings from float32 compression


## Step 6: Create SVS-VAMANA Index

Create the SVS-VAMANA index with the recommended compression settings.

In [9]:
# Create SVS-VAMANA schema with compression
svs_schema = {
    "index": {
        "name": "svs_demo_index",
        "prefix": "demo:svs:",
    },
    "fields": [
        {"name": "movie_id", "type": "tag"},
        {"name": "title", "type": "text"},
        {"name": "genre", "type": "tag"},
        {"name": "rating", "type": "numeric"},
        {"name": "description", "type": "text"},
        {
            "name": "embedding",
            "type": "vector",
            "attrs": {
                "dims": target_dims,  # Use reduced dimensions
                "algorithm": "svs-vamana",
                "datatype": selected_config.datatype,
                "distance_metric": "cosine"
                # Note: Don't include the full selected_config to avoid dims/reduce conflict
            }
        }
    ]
}

print("Creating SVS-VAMANA index with compression...")
svs_index = SearchIndex.from_dict(svs_schema, redis_url=REDIS_URL)
svs_index.create(overwrite=True)
print(
    f"✅ Created SVS-VAMANA index: {svs_index.name}",
    f"Compression: {selected_config.compression if hasattr(selected_config, 'compression') else 'None'}",
    f"Datatype: {selected_config.datatype}",
    f"Dimensions: {dims} → {target_dims}",
    sep="\n"
)

Creating SVS-VAMANA index with compression...
✅ Created SVS-VAMANA index: svs_demo_index
Compression: LVQ4
Datatype: float32
Dimensions: 768 → 768


## Step 7: Migrate Data from HNSW to SVS-VAMANA

Extract data from the HNSW index and migrate it to SVS-VAMANA with compression.

In [10]:
# Extract data from HNSW index
print("🔄 Extracting data from HNSW index...")

client = redis.Redis.from_url(REDIS_URL)
keys = client.keys("demo:hnsw:*")
print(f"Found {len(keys)} documents to migrate")

# Process and transform data for SVS index
svs_data = []

for key in keys:
    doc_data = client.hgetall(key)
    
    if b'embedding' in doc_data:
        # Extract original vector from HNSW index
        original_vector = np.array(buffer_to_array(doc_data[b'embedding'], dtype='float32'))
        
        # Apply dimensionality reduction if needed (LeanVec)
        if target_dims < dims:
            vector = original_vector[:target_dims]
        else:
            vector = original_vector
        
        # Convert to target datatype
        if target_dtype == 'float16':
            vector = vector.astype(np.float16)
        
        svs_data.append({
            "movie_id": doc_data[b'movie_id'].decode(),
            "title": doc_data[b'title'].decode(),
            "genre": doc_data[b'genre'].decode(),
            "rating": int(doc_data[b'rating'].decode()),
            "description": doc_data[b'description'].decode(),
            "embedding": array_to_buffer(vector, dtype=target_dtype)
        })

print(f"Prepared {len(svs_data)} documents for migration")

🔄 Extracting data from HNSW index...
Found 20 documents to migrate
Prepared 20 documents for migration


In [11]:
# Load data into SVS index
print("📥 Loading data into SVS-VAMANA index...")
batch_size = 100  # Define batch size for migration

if len(svs_data) > 0:
    for i in range(0, len(svs_data), batch_size):
        batch = svs_data[i:i+batch_size]
        svs_index.load(batch)
        print(f"  Migrated {min(i+batch_size, len(svs_data))}/{len(svs_data)} documents")

    # Wait for indexing to complete
    print("⏳ Waiting for SVS-VAMANA indexing to complete...")
    time.sleep(5)

    svs_info = svs_index.info()
    print(
        f"\n✅ Migration complete! SVS index has {svs_info['num_docs']} documents",
        f"Index size: {svs_info.get('vector_index_sz_mb', 'N/A')} MB",
        sep="\n"
    )
else:
    print("⚠️  No data to migrate. Make sure the HNSW index was populated first.")
    print("   Run the previous cells to load data into the HNSW index.")
    svs_info = svs_index.info()

📥 Loading data into SVS-VAMANA index...
  Migrated 20/20 documents
⏳ Waiting for SVS-VAMANA indexing to complete...

✅ Migration complete! SVS index has 20 documents
Index size: 3.017791748046875 MB


## Step 8: Compare Memory Usage

Analyze the memory savings achieved through the HNSW to SVS-VAMANA migration.

In [12]:
# Helper function to extract memory info
def get_memory_mb(index_info):
    """Extract memory usage in MB from index info"""
    memory = index_info.get('vector_index_sz_mb', 0)
    if isinstance(memory, str):
        try:
            return float(memory)
        except ValueError:
            return 0.0
    return float(memory)

# Get memory usage
hnsw_memory = get_memory_mb(hnsw_info)
svs_memory = get_memory_mb(svs_info)

print(
    "📊 Memory Usage Comparison",
    "=" * 40,
    f"Original HNSW index:    {hnsw_memory:.2f} MB",
    f"SVS-VAMANA index:       {svs_memory:.2f} MB",
    "",
    sep="\n"
)

if hnsw_memory > 0:
    if svs_memory > 0:
        savings = ((hnsw_memory - svs_memory) / hnsw_memory) * 100
        print(
            f"💰 Memory savings: {savings:.1f}%",
            f"Absolute reduction: {hnsw_memory - svs_memory:.2f} MB",
            sep="\n"
        )
    else:
        print("⏳ SVS index still indexing - memory comparison pending")
else:
    print("⚠️  Memory information not available")

📊 Memory Usage Comparison
Original HNSW index:    3.23 MB
SVS-VAMANA index:       3.02 MB

💰 Memory savings: 6.5%
Absolute reduction: 0.21 MB


## Step 9: Validate Search Quality

Compare search quality between HNSW and SVS-VAMANA to ensure the migration maintains acceptable recall.

In [13]:
# Generate test queries
print("🔍 Generating test queries for quality validation...")

np.random.seed(123)  # For reproducible test queries
num_test_queries = 10
test_queries = []

for i in range(num_test_queries):
    # Create test query vectors
    query_vec = np.random.random(dims).astype(np.float32)
    query_vec = query_vec / np.linalg.norm(query_vec)  # Normalize
    test_queries.append(query_vec)

print(f"Generated {len(test_queries)} test queries")

🔍 Generating test queries for quality validation...
Generated 10 test queries


In [14]:
# Test HNSW search quality
print("🔍 Testing HNSW search quality...")

hnsw_results = []
hnsw_start = time.time()

for query_vec in test_queries:
    query = VectorQuery(
        vector=query_vec,
        vector_field_name="embedding",
        return_fields=["movie_id", "title", "genre"],
        dtype="float32",
        num_results=10
    )
    results = hnsw_index.query(query)
    hnsw_results.append([doc["movie_id"] for doc in results])

hnsw_time = time.time() - hnsw_start
print(f"HNSW search completed in {hnsw_time:.3f} seconds")

🔍 Testing HNSW search quality...
HNSW search completed in 0.010 seconds


In [15]:
# Test SVS-VAMANA search quality
print("🔍 Testing SVS-VAMANA search quality...")

svs_results = []
svs_start = time.time()

for i, query_vec in enumerate(test_queries):
    # Adjust query vector for SVS index (handle dimensionality reduction)
    if target_dims < dims:
        svs_query_vec = query_vec[:target_dims]
    else:
        svs_query_vec = query_vec
    
    if target_dtype == 'float16':
        svs_query_vec = svs_query_vec.astype(np.float16)
    
    query = VectorQuery(
        vector=svs_query_vec,
        vector_field_name="embedding",
        return_fields=["movie_id", "title", "genre"],
        dtype=target_dtype,
        num_results=10
    )
    results = svs_index.query(query)
    svs_results.append([doc["movie_id"] for doc in results])

svs_time = time.time() - svs_start
print(f"SVS-VAMANA search completed in {svs_time:.3f} seconds")

🔍 Testing SVS-VAMANA search quality...
SVS-VAMANA search completed in 0.009 seconds


In [16]:
# Calculate recall and performance metrics
def calculate_recall(reference_results, test_results, k=10):
    """Calculate recall@k between two result sets"""
    if not reference_results or not test_results:
        return 0.0
    
    total_recall = 0.0
    for ref, test in zip(reference_results, test_results):
        ref_set = set(ref[:k])
        test_set = set(test[:k])
        if len(ref_set) > 0:
            recall = len(ref_set.intersection(test_set)) / len(ref_set)
            total_recall += recall
    
    return total_recall / len(reference_results)

# Calculate metrics
recall_at_5 = calculate_recall(hnsw_results, svs_results, k=5)
recall_at_10 = calculate_recall(hnsw_results, svs_results, k=10)

print(
    "📊 Search Quality Comparison",
    "=" * 40,
    "HNSW (baseline):        100% recall (exact graph-based search)",
    f"SVS-VAMANA Recall@5:    {recall_at_5*100:.1f}% (vs HNSW baseline)",
    f"SVS-VAMANA Recall@10:   {recall_at_10*100:.1f}% (vs HNSW baseline)",
    "",
    "⏱️ Performance Comparison:",
    f"HNSW query time:        {hnsw_time:.3f}s ({hnsw_time/num_test_queries*1000:.1f}ms per query)",
    f"SVS-VAMANA query time:  {svs_time:.3f}s ({svs_time/num_test_queries*1000:.1f}ms per query)",
    f"Speed difference:       {((hnsw_time - svs_time) / hnsw_time * 100):+.1f}%",
    sep="\n"
)

# Quality assessment
if recall_at_10 >= 0.95:
    quality_assessment = "🟢 Excellent - Minimal quality loss"
elif recall_at_10 >= 0.90:
    quality_assessment = "🟡 Good - Acceptable quality for most applications"
elif recall_at_10 >= 0.80:
    quality_assessment = "🟠 Fair - Consider if quality requirements are flexible"
else:
    quality_assessment = "🔴 Poor - Migration not recommended"

print(f"\n🎯 Quality Assessment: {quality_assessment}")

📊 Search Quality Comparison
HNSW (baseline):        100% recall (exact graph-based search)
SVS-VAMANA Recall@5:    100.0% (vs HNSW baseline)
SVS-VAMANA Recall@10:   100.0% (vs HNSW baseline)

⏱️ Performance Comparison:
HNSW query time:        0.010s (1.0ms per query)
SVS-VAMANA query time:  0.009s (0.9ms per query)
Speed difference:       +5.9%

🎯 Quality Assessment: 🟢 Excellent - Minimal quality loss


## Step 10: Migration Decision Framework

Based on the analysis, determine if migration is recommended.

In [17]:
# Migration decision logic
memory_savings_threshold = 5  # Minimum % memory savings
recall_threshold = 0.85  # Minimum 85% recall@10

memory_savings_pct = ((hnsw_memory - svs_memory) / hnsw_memory * 100) if hnsw_memory > 0 and svs_memory > 0 else 0
meets_memory_threshold = memory_savings_pct >= memory_savings_threshold
meets_quality_threshold = recall_at_10 >= recall_threshold

print(
    "🤔 Migration Decision Analysis",
    "=" * 40,
    "",
    "📊 Criteria Evaluation:",
    f"Memory savings: {memory_savings_pct:.1f}% {'✅' if meets_memory_threshold else '❌'} (threshold: {memory_savings_threshold}%)",
    f"Search quality: {recall_at_10:.3f} {'✅' if meets_quality_threshold else '❌'} (threshold: {recall_threshold})",
    "",
    sep="\n"
)

if meets_memory_threshold and meets_quality_threshold:
    recommendation = "🟢 RECOMMENDED"
    reasoning = "Migration provides significant memory savings while maintaining good search quality."
elif meets_memory_threshold and not meets_quality_threshold:
    recommendation = "🟡 CONDITIONAL"
    reasoning = "Good memory savings but reduced search quality. Consider if your application can tolerate lower recall."
elif not meets_memory_threshold and meets_quality_threshold:
    recommendation = "🟠 LIMITED BENEFIT"
    reasoning = "Search quality is maintained but memory savings are minimal. Migration may not be worth the effort."
else:
    recommendation = "🔴 NOT RECOMMENDED"
    reasoning = "Insufficient memory savings and/or poor search quality. Consider alternative optimization strategies."

print(
    f"🎯 Migration Recommendation: {recommendation}",
    f"💭 Reasoning: {reasoning}",
    sep="\n"
)

🤔 Migration Decision Analysis

📊 Criteria Evaluation:
Memory savings: 6.5% ✅ (threshold: 5%)
Search quality: 1.000 ✅ (threshold: 0.85)

🎯 Migration Recommendation: 🟢 RECOMMENDED
💭 Reasoning: Migration provides significant memory savings while maintaining good search quality.


## Step 12: Cleanup

Clean up the demonstration indices.

In [18]:
print("🧹 Cleaning up demonstration indices...")

# Clean up HNSW index
try:
    hnsw_index.delete(drop=True)
    print("✅ Deleted HNSW demonstration index")
except Exception as e:
    print(f"⚠️  Failed to delete HNSW index: {e}")

# Clean up SVS index
try:
    svs_index.delete(drop=True)
    print("✅ Deleted SVS-VAMANA demonstration index")
except Exception as e:
    print(f"⚠️  Failed to delete SVS index: {e}")

print(
    "\n🎉 HNSW to SVS-VAMANA migration demonstration complete!",
    "\nNext steps:",
    "1. Apply learnings to your production HNSW indices",
    "2. Test with your actual query patterns and data",
    "3. Monitor performance in your environment",
    "4. Consider gradual rollout strategy",
    "5. Evaluate impact on applications using HNSW-specific features",
    sep="\n"
)

🧹 Cleaning up demonstration indices...
✅ Deleted HNSW demonstration index
✅ Deleted SVS-VAMANA demonstration index

🎉 HNSW to SVS-VAMANA migration demonstration complete!

Next steps:
1. Apply learnings to your production HNSW indices
2. Test with your actual query patterns and data
3. Monitor performance in your environment
4. Consider gradual rollout strategy
5. Evaluate impact on applications using HNSW-specific features
